<center><h1>HSE FCS SE ML</h1></center>
<center><h1>"Fast sold post prediction" Kaggle competition</h1></center>
<center><h2>Team name: Turbo 3D</h2></center>
<center><h3>Daniil Kraynov, Dmitry Strokov, Danil Kolesnikov</h3></center>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

# Read Data

In [ ]:
df = pd.read_csv('train.tsv', sep='\t').drop(columns='Unnamed: 0')

In [ ]:
df.head()

In [ ]:
df.info()

# Prepare / Encode Data

In [ ]:
# Some checks to see what columns should not be considered
print(any(df['product_id'].duplicated()))

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import time
import datetime

def preprocess(store_data):
    lb = LabelEncoder()
    #ohe = OneHotEncoder()
    
    s = store_data.drop(columns=['product_id', 'name_text', 'desc_text', 'properties'])

    # Delivery/payment encoding: 0 is false (bad), 1 is true (good)
    s['delivery_available'] = lb.fit_transform(s['delivery_available'])
    s['payment_available'] = lb.fit_transform(s['payment_available'])
    # City/region encoding
    s['city'] = lb.fit_transform(s['city'])
    s['region'] = lb.fit_transform(s['region'])
    # Converting date to timestamp
    dates = s['date_created'].values
    convert_time = lambda d: int(time.mktime(datetime.datetime.strptime(d, "%Y-%m-%d").timetuple()))
    s['date_created'] = list(map(convert_time, dates))
    # Misc
    s['owner_id'] = lb.fit_transform(s['owner_id'])
    
    # Categorical features encoding
    # Бля получается для каждой категории своя колонка будет? Все ж упадет нахуй тогда
    # categorical_cols = ['category_id', 'city', 'region', 'subcategory_id']
    # store_data = pd.get_dummies(store_data, columns=categorical_cols)
    
    return s

In [ ]:
df_p = preprocess(df)

In [ ]:
df_p.head()

In [ ]:
df_p.info()

In [ ]:
X = df_p.drop(columns=['sold_fast'])
y = df_p['sold_fast']

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Some Assumptions / Graphs

In [ ]:
for feature in ('delivery_available', 'payment_available', 'img_num', 'price'):
    print(feature, X[feature].corr(y))

# Fitting & Predicting

In [ ]:
# To avoid kernel dying
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
import xgboost as xgb

In [ ]:
from sklearn import metrics
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

def modelfit(alg, Xt, yt, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(Xt, label=yt)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(Xt, yt, eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(Xt)
    dtrain_predprob = alg.predict_proba(Xt)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(yt, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(yt, dtrain_predprob))
                    
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
def do_gridsearch(static_params, search_params):
    gsearch = GridSearchCV(
        estimator = XGBClassifier(**static_params),
        param_grid = search_params,
        scoring='roc_auc',
        n_jobs=4,
        iid=False,
        cv=5,
        verbose=3
    )
    gsearch.fit(X, y)
    return gsearch.best_params_, gsearch.best_score_

In [ ]:
xgb_params = {
    'base_score': 0.5,
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'colsample_bylevel': 1,
    'gamma': 0.46,
    'learning_rate': 0.1,
    'n_estimators': 444,
    'max_depth': 4,
    'min_child_weight': 4,
    'max_delta_step': 0,
    'missing': None,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'seed': 27,
    'silent': 1,
    'subsample': 0.95,
    'colsample_bytree': 0.8,
    'nthread': 4,
    'n_jobs': 4
}

search_params = {
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100]
}

#bp, bs = do_gridsearch(xgb_params, search_params)
#bp, bs

xgbm = XGBClassifier(**xgb_params)
modelfit(xgbm, X, y)

In [ ]:
from sklearn.model_selection import cross_val_score

model = XGBClassifier(**xgb_params)

cross_val_score(model, X, y, cv=5, scoring='roc_auc', verbose=3, n_jobs=4)

In [ ]:
model = XGBClassifier()
cross_val_score(model, X, y, cv=5, scoring='roc_auc', verbose=3, n_jobs=4)

# Test Data Prediction

In [ ]:
df_test = pd.read_csv('test_nolabel.tsv', sep='\t').drop(columns='Unnamed: 0')
df_test["sold_fast"] = 0
df_merged = pd.concat([df, df_test])

In [ ]:
df_merged_p = preprocess(df_merged)
df_mp = df_merged_p[:len(df)]
df_test_mp = df_merged_p[len(df):]

In [ ]:
X_fit = df_mp.drop(columns=['sold_fast'])
y_fit = df_mp['sold_fast']
X_actual = df_test_mp.drop(columns=['sold_fast'])

In [ ]:
model = XGBClassifier(n_estimators=1000, learning_rate=0.02, max_depth=8, n_jobs=3, colsample_bytree=0.7, scale_pos_weight=1.)
model.fit(X_fit, y_fit)
y_actual = model.predict_proba(X_actual)[:, 1]
y_actual

In [ ]:
df_final = pd.DataFrame.from_dict({'product_id' : df_test['product_id'].values, 'score' : y_actual})
df_final.to_csv('submission.csv', sep = ',', index = False)